In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
import  pyspark.sql.functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import StringType

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1664671886566_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# specify input-data and output_data
input_data = "s3a://udacity-dend/log_data"
output_data = "s3://thanhdv-spark-datalake-emr-project4-03/emr-lab-data/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.builder.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0").getOrCreate()
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
### process_log_data ###

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# For dev and test faster
log_data = "s3://udacity-dend/log_data/2018/11/2018-11-12-events.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = spark.read.json(log_data)
df = df.filter(df.page == 'NextSong')
df.printSchema()
df.show(10)
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

+--------------------+---------+----------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth| firstName|gender|itemInSession|lastName|   le

In [6]:
# extract columns for users table  
users_table = df.select("userId","firstName","lastName","gender","level").dropDuplicates()
users_table.printSchema()
users_table.sort("userId").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

+------+----------+---------+------+-----+
|userId|firstName |lastName |gender|level|
+------+----------+---------+------+-----+
|10    |Sylvie    |Cruz     |F     |free |
|100   |Adler     |Barrera  |M     |free |
|12    |Austin    |Rosales  |M     |free |
|14    |Theodore  |Harris   |M     |free |
|26    |Ryan      |Smith    |M     |free |
|29    |Jacqueline|Lynch    |F     |paid |
|32    |Lily      |Burns    |F     |free |
|33    |Bronson   |Harris   |M     |free |
|34    |Evelin    |Ayala    |F     |free |
|37    |Jordan    |Hicks    |F     |free |
|49    |Chloe     |Cuevas   |F     |free |
|51    |Maia      |Burke    |F     |free |
|53    |Celeste   |Williams |F     |free |
|58    |Emily     |Benson   |F     |paid |
|61    |Samuel    |Gonzalez |M     |free |
|66    |Kevin     |Arellan

In [7]:
# write users table to parquet files
users_table.write.parquet(os.path.join(output_data, "users/users.parquet"), "overwrite")

# verify parquet files
users_df = spark.read.parquet("s3://thanhdv-spark-datalake-emr-project4-03/emr-lab-data/users/users.parquet/*")
users_df.printSchema()
users_df.sort("userId").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

+------+----------+---------+------+-----+
|userId|firstName |lastName |gender|level|
+------+----------+---------+------+-----+
|10    |Sylvie    |Cruz     |F     |free |
|100   |Adler     |Barrera  |M     |free |
|12    |Austin    |Rosales  |M     |free |
|14    |Theodore  |Harris   |M     |free |
|26    |Ryan      |Smith    |M     |free |
|29    |Jacqueline|Lynch    |F     |paid |
|32    |Lily      |Burns    |F     |free |
|33    |Bronson   |Harris   |M     |free |
|34    |Evelin    |Ayala    |F     |free |
|37    |Jordan    |Hicks    |F     |free |
|49    |Chloe     |Cuevas   |F     |free |
|51    |Maia      |Burke    |F     |free |
|53    |Celeste   |Williams |F     |free |
|58    |Emily     |Benson   |F     |paid |
|61    |Samuel    |Gonzalez |M     |free |
|66    |Kevin     |Arellan

In [8]:
# Verify schema differences between parquet file and dataframe
users_table = users_table.filter(users_table.level == 'paid')
users_table.printSchema()
users_table.sort("userId").show(truncate=False)

users_df = users_df.filter(users_df.level == 'paid')
users_df.printSchema()
users_df.sort("userId").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

+------+----------+--------+------+-----+
|userId|firstName |lastName|gender|level|
+------+----------+--------+------+-----+
|29    |Jacqueline|Lynch   |F     |paid |
|58    |Emily     |Benson  |F     |paid |
|73    |Jacob     |Klein   |M     |paid |
|80    |Tegan     |Levine  |F     |paid |
|97    |Kate      |Harrell |F     |paid |
+------+----------+--------+------+-----+

root
 |-- userId: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

+------+----------+--------+------+-----+
|userId|firstName |lastName|gender|level|
+------+----------+--------+------+-----+
|29    |Jacqueline|Lynch   |F     |paid |
|58    |Emily     |Benson  |F     |paid |
|73    |Jac

In [9]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x:  datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
df = df.withColumn("timestamp", get_timestamp(df.ts))

df.printSchema()
df.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: string (nullable = true)

+--------------------+---------+----------+------+-------------+---------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+
|              a

In [10]:
# create datetime column from original timestamp column
get_datetime = udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d'))
df = df.withColumn("datetime", get_datetime(df.ts))

df.printSchema()
df.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- datetime: string (nullable = true)

+--------------------+---------+----------+------+-------------+---------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+----

In [11]:
# extract columns to create time table
time_table = df.select(col('timestamp').alias('start_time'),
                       hour('timestamp').alias('hour'),
                       dayofmonth('timestamp').alias('day'),
                       weekofyear('timestamp').alias('week'),
                       month('timestamp').alias('month'),
                       year('timestamp').alias('year'),
                       date_format('timestamp','E').alias('weekday')
)

time_table.printSchema()
time_table.sort("start_time").show(15)
# time_table.sort("start_time").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: string (nullable = true)

+-------------------+----+---+----+-----+----+-------+
|         start_time|hour|day|week|month|year|weekday|
+-------------------+----+---+----+-----+----+-------+
|2018-11-12 02:37:38|   2| 12|  46|   11|2018|    Mon|
|2018-11-12 02:37:44|   2| 12|  46|   11|2018|    Mon|
|2018-11-12 02:42:21|   2| 12|  46|   11|2018|    Mon|
|2018-11-12 02:45:52|   2| 12|  46|   11|2018|    Mon|
|2018-11-12 02:47:22|   2| 12|  46|   11|2018|    Mon|
|2018-11-12 02:50:21|   2| 12|  46|   11|2018|    Mon|
|2018-11-12 02:54:26|   2| 12|  46|   11|2018|    Mon|
|2018-11-12 02:57:12|   2| 12|  46|   11|2018|    Mon|
|2018-11-12 03:00:48|   3| 12|  46|   11|2018|    Mon|
|2018-11-12 03:03:24|   3| 12|  46|   11|2018|    Mon|
|2018-11-12 0

In [12]:
# write time table to parquet files partitioned by year and month
time_table.write.partitionBy("year", "month").parquet(os.path.join(output_data, "time/time.parquet"), "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Verify time_table parquet file
time_table = spark.read.parquet("s3://thanhdv-spark-datalake-emr-project4-03/emr-lab-data/time/time.parquet/*")

time_table.printSchema()
time_table.sort("start_time").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- month: integer (nullable = true)

+-------------------+----+---+----+-------+-----+
|start_time         |hour|day|week|weekday|month|
+-------------------+----+---+----+-------+-----+
|2018-11-12 02:37:38|2   |12 |46  |Mon    |11   |
|2018-11-12 02:37:44|2   |12 |46  |Mon    |11   |
|2018-11-12 02:42:21|2   |12 |46  |Mon    |11   |
|2018-11-12 02:45:52|2   |12 |46  |Mon    |11   |
|2018-11-12 02:47:22|2   |12 |46  |Mon    |11   |
|2018-11-12 02:50:21|2   |12 |46  |Mon    |11   |
|2018-11-12 02:54:26|2   |12 |46  |Mon    |11   |
|2018-11-12 02:57:12|2   |12 |46  |Mon    |11   |
|2018-11-12 03:00:48|3   |12 |46  |Mon    |11   |
|2018-11-12 03:03:24|3   |12 |46  |Mon    |11   |
|2018-11-12 03:36:28|3   |12 |46  |Mon    |11   |
|2018-11-12 06:13:27|6   |12 |46  |Mon    |11   |
|2018-11-12 08:

In [14]:
# extract columns from joined song and log datasets to create songplays table 
# defined log_data = input_data + "song_data/A/A/A/*.json"
songplays_list = df.withColumn("timestamp", get_timestamp(df.ts))
songplays_list.createOrReplaceTempView("log_data_table")
#songplays_list.printSchema()
songplays_list.sort("ts").show(truncate=True)

song_df = spark.read.json(log_data)
song_df.createOrReplaceTempView("song_data_table")
#song_df.printSchema()
song_df.sort("ts").show(truncate=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+----------+------+-------------+---------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+----------+
|              artist|     auth| firstName|gender|itemInSession| lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|          timestamp|  datetime|
+--------------------+---------+----------+------+-------------+---------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+-------------------+----------+
|            Pavement|Logged In|    Sylvie|     F|            0|     Cruz| 99.16036| free|Washington-Arling...|   PUT|NextSong|1.540266185796E12|      345|Mercy:The Laundromat|   200|1541990258796|"Mozilla/5.0 (Mac...|    10|2018

In [16]:
songplays_table = spark.sql("""
    SELECT
        ldt.ts as start_time,
        ldt.userId AS user_id,
        ldt.level as level,
        sdt.song_id as song_id,
        sdt.artist_id as artist_id,
        ldt.sessionId as session_id,
        ldt.location as location,
        ldt.userAgent as user_agent,
        year(ldt.timestamp) as year,
        month(ldt.timestamp) as month,
        row_number() as songplay_id
    FROM log_data_table ldt
    JOIN 
        song_data_table sdt ON ldt.song = sdt.song AND ldt.artist = sdt.artist
    """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"cannot resolve '`sdt.song_id`' given input columns: [sdt.artist, sdt.auth, ldt.userId, sdt.sessionId, ldt.lastName, ldt.sessionId, ldt.status, sdt.level, ldt.auth, ldt.gender, sdt.method, ldt.length, sdt.status, sdt.length, ldt.registration, sdt.lastName, sdt.song, sdt.userId, sdt.location, ldt.itemInSession, ldt.ts, ldt.location, ldt.page, ldt.song, ldt.datetime, ldt.level, ldt.artist, sdt.page, sdt.itemInSession, sdt.gender, sdt.firstName, ldt.firstName, ldt.userAgent, sdt.ts, sdt.registration, ldt.method, sdt.userAgent, ldt.timestamp]; line 6 pos 8;\n'Project [ts#21L AS start_time#763L, userId#23 AS user_id#764, level#13 AS level#765, 'sdt.song_id AS song_id#766, 'sdt.artist_id AS artist_id#767, sessionId#18L AS session_id#768L, location#14 AS location#769, userAgent#22 AS user_agent#770, year(cast(timestamp#535 as date)) AS year#771, month(cast(timestamp#535 as date)) AS month#772, row_number() AS songplay_id#773]\n+- Join Inner, ((song#19 = song#656) && (artist#6 = artist#643))\n

In [ ]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.partitionBy("year", "month").parquet(os.path.join(output_data, "songplays/songplays.parquet"), "overwrite")